## Using OpenCV and a webcam to visualize the mediapipe model we will use

In [3]:
import mediapipe as mp
import pandas as pd
import cv2
import os
import numpy as np
import csv
from sklearn.model_selection import train_test_split

In [4]:
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic 

In [5]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = holistic.process(image)


        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                  mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                  )

        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                                  )

        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                                  )

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        cv2.imshow('Action Recognition', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## Setting up the formatting for the landmark dataframe (.csv/excel)

In [6]:
num_coords = len(results.pose_landmarks.landmark) + len(results.face_landmarks.landmark)

In [7]:
landmarks = ['class']
for num in range(1, num_coords+1):
    landmarks += ['x{}'.format(num), 'y{}'.format(num), 'z{}'.format(num), 'v{}'.format(num)]

## Creating the CSV file

In [10]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

## Declaring name of the class you are about to extract landmarks for. You will change this as you add more actions

In [11]:
class_name = "Smile"

## Using webcam to extract landmarks

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = holistic.process(image)


        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                  mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                  )

        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                                  )

        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                                  )

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )
        
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            row = pose_row + face_row
            row.insert(0, class_name)
            
            with open('coords_3.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
        except:
            pass

        cv2.imshow('Action Recognition', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## Setting up dataframe using pandas

In [12]:
df = pd.read_csv('coords.csv')

## Next 3 cells are just visualizing aspects of the dataframe

In [13]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Smile,0.636343,0.261224,-0.890196,0.999741,0.665204,0.211376,-0.856812,0.999393,0.681240,...,-0.007901,0.0,0.695879,0.220290,0.006280,0.0,0.700221,0.216940,0.006543,0.0
1,Smile,0.636846,0.261206,-0.914669,0.999739,0.665125,0.211604,-0.879333,0.999395,0.681217,...,-0.007499,0.0,0.695620,0.220091,0.007166,0.0,0.699989,0.216234,0.007518,0.0
2,Smile,0.637636,0.261496,-0.954743,0.999738,0.665170,0.212875,-0.919594,0.999397,0.681239,...,-0.007496,0.0,0.694897,0.220624,0.007476,0.0,0.699280,0.216849,0.007850,0.0
3,Smile,0.638423,0.261728,-0.953200,0.999739,0.665338,0.213974,-0.917990,0.999402,0.681347,...,-0.007378,0.0,0.695029,0.221161,0.007239,0.0,0.699432,0.217174,0.007607,0.0
4,Smile,0.638855,0.262032,-0.956315,0.999743,0.665478,0.214917,-0.920459,0.999414,0.681447,...,-0.007041,0.0,0.694534,0.220839,0.007396,0.0,0.698922,0.216830,0.007778,0.0


In [14]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
2271,Smile,0.612959,0.283884,-1.138359,0.999889,0.640400,0.225654,-1.083799,0.999773,0.658737,...,-0.011228,0.0,0.684069,0.240236,0.006916,0.0,0.688752,0.234218,0.007578,0.0
2272,Smile,0.614091,0.286355,-1.138892,0.999887,0.641281,0.227579,-1.085648,0.999772,0.659383,...,-0.013035,0.0,0.686181,0.245377,0.003848,0.0,0.690927,0.239153,0.004401,0.0
2273,Smile,0.614711,0.294068,-1.147451,0.999888,0.642066,0.234803,-1.098770,0.999774,0.659965,...,-0.016623,0.0,0.689315,0.255477,0.000725,0.0,0.693799,0.250299,0.001157,0.0
2274,Smile,0.615414,0.301934,-1.156046,0.999886,0.642789,0.241015,-1.110079,0.999771,0.660477,...,-0.016864,0.0,0.687962,0.258790,-0.000533,0.0,0.692419,0.253456,-0.000178,0.0
2275,Smile,0.615988,0.305764,-1.157575,0.999884,0.643363,0.244249,-1.111927,0.999769,0.660897,...,-0.017120,0.0,0.688262,0.258566,-0.001319,0.0,0.692738,0.253135,-0.001022,0.0


In [15]:
df[df['class']=='Neutral']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1216,Neutral,0.614732,0.261171,-0.744511,0.999932,0.643117,0.208318,-0.708958,0.999857,0.659539,...,-0.008200,0.0,0.679957,0.224952,0.009127,0.0,0.683642,0.221529,0.009383,0.0
1217,Neutral,0.614652,0.262937,-0.939952,0.999929,0.642343,0.210781,-0.893266,0.999853,0.659058,...,-0.008367,0.0,0.681219,0.225039,0.008717,0.0,0.685066,0.221456,0.008913,0.0
1218,Neutral,0.614238,0.264707,-0.981592,0.999924,0.641736,0.212792,-0.939283,0.999842,0.658759,...,-0.008116,0.0,0.680810,0.226006,0.008620,0.0,0.684697,0.222525,0.008760,0.0
1219,Neutral,0.614219,0.265774,-1.002040,0.999922,0.641550,0.213822,-0.957838,0.999841,0.658665,...,-0.007937,0.0,0.680482,0.226436,0.009068,0.0,0.684343,0.222865,0.009235,0.0
1220,Neutral,0.614221,0.266565,-1.023901,0.999921,0.641500,0.214512,-0.977464,0.999842,0.658643,...,-0.008017,0.0,0.680468,0.225924,0.008755,0.0,0.684319,0.222442,0.008898,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,Neutral,0.616383,0.273621,-1.130857,0.999892,0.643192,0.218152,-1.081913,0.999799,0.661930,...,-0.005785,0.0,0.680819,0.226318,0.011236,0.0,0.685019,0.222477,0.011430,0.0
1881,Neutral,0.613857,0.273548,-1.119342,0.999892,0.640671,0.218148,-1.071565,0.999798,0.659602,...,-0.006457,0.0,0.679089,0.225480,0.010001,0.0,0.683310,0.221795,0.010116,0.0
1882,Neutral,0.612322,0.273502,-1.117055,0.999892,0.639135,0.218145,-1.069430,0.999797,0.658158,...,-0.006666,0.0,0.678964,0.225540,0.009736,0.0,0.683158,0.221817,0.009860,0.0
1883,Neutral,0.611498,0.273364,-1.134248,0.999892,0.638276,0.218133,-1.089597,0.999796,0.657393,...,-0.006369,0.0,0.678553,0.225483,0.009995,0.0,0.682745,0.221757,0.010122,0.0


## Separating the target variable from the input features

In [16]:
X = df.drop('class', axis=1)
y = df['class']

In [17]:
y

0       Smile
1       Smile
2       Smile
3       Smile
4       Smile
        ...  
2271    Smile
2272    Smile
2273    Smile
2274    Smile
2275    Smile
Name: class, Length: 2276, dtype: object

## Setting up the test/train splits for the training process. As shown in the code, we will be using a split of 30% (30% of the training data will be held out for testing)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234) 

In [19]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

## Setting up our pipeline for the different algorithms we will train for this task. You can add/replace these with different algorithms if you'd like

In [20]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rfc':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gbc':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [21]:
list(pipelines.values())[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

## Training (Fitting) the models that are declared in our pipeline

In [22]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train.values, y_train.values) #added .values 
    fit_models[algo] = model

c:\users\oshan\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rfc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gbc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [24]:
from sklearn.metrics import accuracy_score
import pickle

## Displaying the basic accuracy metric for our trained models. You can import many other performance metrics if you'd like

In [25]:
for algo, model in fit_models.items():
    y_hat = model.predict(X_test.values)
    print(algo, accuracy_score(y_test, y_hat))

lr 1.0
rc 1.0
rfc 1.0
gbc 1.0


## Saving our model

In [37]:
with open('Action_Recognition_gbc.pkl', 'wb') as f:
    pickle.dump(fit_models['gbc'], f)

## Loading our model

In [39]:
with open('Action_Recognition_gbc.pkl', 'rb') as f:
    model = pickle.load(f)

In [32]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

## Using webcam for real-time detections

### Heads-up: For the 

In [40]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = holistic.process(image)


        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                  mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                  )

        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                                  )

        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                                  )

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )
        
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            row = pose_row + face_row
            
#             row.insert(0, class_name)
            
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row)

            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            
#             coords = tuple(np.multiply(
#                             np.array(
#                                     ( results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
#                                       results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
#                             , [640, 480]).astype(int))
            
#             cv2.rectangle(image, 
#                           (coords[0], coords[1]+5), 
#                           (coords[0]+len(body_language_class)*20, coords[1]-30), 
#                           (245, 117, 16), -1)
#             cv2.putText(image, body_language_class, coords, 
#                         cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
            
            
        except:
            pass

        cv2.imshow('Action Recognition', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Sad [0.1546722  0.52087722 0.22395188 0.1004987 ]
Sad [0.13084349 0.59479097 0.18944998 0.08491556]
Sad [0.132677   0.58901126 0.19210474 0.086207  ]
Sad [0.11935469 0.6303706  0.17281521 0.0774595 ]
Sad [0.10993849 0.65953162 0.15918137 0.07134852]
Sad [0.11740245 0.60620403 0.20011117 0.07628235]
Sad [0.10692009 0.64144591 0.18224411 0.06938988]
Sad [0.10691135 0.64139348 0.18222921 0.06946596]
Sad [0.12070008 0.59514275 0.20573195 0.07842522]
Sad [0.1283217  0.56967619 0.21872291 0.08327921]
Sad [0.15368278 0.46562007 0.27406333 0.10663381]
Sad [0.15323326 0.46673472 0.27326169 0.10677033]
Sad [0.12084235 0.5798387  0.21549881 0.08382015]
Sad [0.1006631  0.65030916 0.17951304 0.0695147 ]
Sad [0.10089321 0.64845295 0.1799234  0.07073045]
Sad [0.07024729 0.52455411 0.12527236 0.27992624]
Sad [0.06258216 0.57643302 0.1116031  0.24938171]
Sad [0.06258216 0.57643302 0.1116031  0.24938171]
Sad [0.06258216 0.57643302 0.1116031  0.24938171]
Sad [0.05807206 0.60695827 0.10356022 0.23140945]


Smile [0.23156463 0.11373595 0.48745542 0.16724401]
Smile [0.23138333 0.11376612 0.48758474 0.16726581]
Smile [0.35011914 0.09618874 0.41225071 0.14144141]
Smile [0.35012803 0.09619119 0.41226118 0.1414196 ]
Smile [0.31379502 0.19116135 0.36948057 0.12556306]
Sad [0.14339448 0.45294775 0.30185273 0.10180503]
Sad [0.11046345 0.57848619 0.23253123 0.07851913]
Sad [0.06888025 0.73711834 0.14499645 0.04900497]
Sad [0.03673629 0.85982663 0.07733177 0.02610532]
Sad [0.01276861 0.95130372 0.02687859 0.00904908]
Sad [0.0110651  0.95780074 0.02329261 0.00784155]
Sad [0.00828545 0.96840146 0.01744129 0.0058718 ]
Sad [0.00828544 0.96840053 0.01744128 0.00587275]
Sad [0.00828467 0.96831086 0.01743966 0.00596481]
Sad [0.00828463 0.96830611 0.01743958 0.00596968]
Sad [0.01294179 0.95048959 0.02724313 0.0093255 ]
Sad [0.01276673 0.95116313 0.02687461 0.00919553]
Sad [0.04121199 0.84230508 0.08675336 0.02972957]
Smile [9.68267293e-08 1.47260590e-06 9.99998361e-01 6.90840721e-08]
Smile [9.68267061e-08 

Smile [1.21584740e-06 5.71357516e-08 9.99998636e-01 9.08404719e-08]
Smile [1.21584740e-06 5.71357516e-08 9.99998636e-01 9.08404719e-08]
Smile [1.21584740e-06 5.68002120e-08 9.99998637e-01 9.08404719e-08]
Smile [1.21584740e-06 5.68002120e-08 9.99998637e-01 9.08404719e-08]
Smile [1.21584740e-06 5.68002120e-08 9.99998637e-01 9.08404719e-08]
Neutral [5.11107442e-01 2.77692347e-08 4.88892486e-01 4.44112847e-08]
Neutral [5.10871596e-01 2.77826308e-08 4.89128331e-01 4.44327090e-08]
Neutral [5.10871596e-01 2.77826308e-08 4.89128331e-01 4.44327090e-08]
Neutral [5.10871596e-01 2.77826308e-08 4.89128331e-01 4.44327090e-08]
Neutral [5.10871596e-01 2.77826308e-08 4.89128331e-01 4.44327090e-08]
Neutral [5.10871596e-01 2.77826308e-08 4.89128331e-01 4.44327090e-08]
Neutral [5.10871596e-01 2.77826308e-08 4.89128331e-01 4.44327090e-08]
Neutral [5.10871596e-01 2.77826308e-08 4.89128331e-01 4.44327090e-08]
Neutral [5.10871596e-01 2.79467530e-08 4.89128331e-01 4.44327090e-08]
Neutral [5.10871596e-01 2.7946

Wave [3.49389361e-07 1.72620824e-07 4.73100494e-07 9.99999005e-01]
Wave [3.49389361e-07 1.72620824e-07 4.73100494e-07 9.99999005e-01]
Wave [2.14802082e-03 1.72250091e-07 4.72084430e-07 9.97851335e-01]
Wave [5.83584483e-03 1.71613496e-07 4.70339718e-07 9.94163513e-01]
Wave [5.17918094e-04 1.72531481e-07 4.72855632e-07 9.99481437e-01]
Wave [1.89989798e-04 1.72588088e-07 4.73010775e-07 9.99809365e-01]
Wave [3.49389361e-07 1.72620824e-07 4.73100494e-07 9.99999005e-01]
Wave [3.49389361e-07 1.72620824e-07 4.73100494e-07 9.99999005e-01]
Wave [3.49389361e-07 1.72620824e-07 4.73100494e-07 9.99999005e-01]
Wave [3.49389361e-07 1.72620824e-07 4.73100494e-07 9.99999005e-01]
Wave [3.49389361e-07 1.72620824e-07 4.73100494e-07 9.99999005e-01]
Wave [3.49389361e-07 1.72620824e-07 4.73100494e-07 9.99999005e-01]
Wave [3.49389361e-07 1.72620824e-07 4.73100494e-07 9.99999005e-01]
Wave [5.17918094e-04 1.72531481e-07 4.72855632e-07 9.99481437e-01]
Wave [5.17918094e-04 1.72531481e-07 4.72855632e-07 9.99481437e

Smile [3.27007849e-07 8.89614557e-07 9.99998525e-01 2.58378642e-07]
Smile [3.27007849e-07 8.89614557e-07 9.99998525e-01 2.58378642e-07]
Smile [3.27007890e-07 7.65672207e-07 9.99998649e-01 2.58378674e-07]
Smile [3.74861733e-07 4.89498643e-07 9.99998971e-01 1.64705816e-07]
Smile [3.22646503e-07 4.21315411e-07 9.99999114e-01 1.41763618e-07]
Smile [4.26735056e-07 3.61674227e-07 9.99999090e-01 1.21695702e-07]
Smile [3.76044361e-07 2.67327898e-07 9.99999266e-01 9.01543597e-08]
Smile [3.76044375e-07 2.29805100e-07 9.99999304e-01 9.01543631e-08]
Smile [3.23597765e-07 1.97754365e-07 9.99999401e-01 7.75806057e-08]
Smile [3.23597754e-07 2.30043890e-07 9.99999369e-01 7.75806032e-08]
Smile [3.23597789e-07 1.24278050e-07 9.99999475e-01 7.75806114e-08]
Smile [3.23597765e-07 1.97754365e-07 9.99999401e-01 7.75806057e-08]
Smile [3.23597789e-07 1.24278050e-07 9.99999475e-01 7.75806114e-08]
Smile [3.23597789e-07 1.24278050e-07 9.99999475e-01 7.75806114e-08]
Smile [4.87032671e-07 1.24278030e-07 9.99999311e

## Same as previous cell, except here we commented out the landmarks and will only show you the prediction and accuracy

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = holistic.process(image)


        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
#                                   mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
#                                   mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
#                                   )

#         mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                                   mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
#                                   mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
#                                   )

#         mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                                   mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
#                                   mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
#                                   )

#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
#                                   mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
#                                   mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
#                                   )
        
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            row = pose_row + face_row
            


            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]

            
            
            
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
            
            
        except:
            pass

        cv2.imshow('Action Recognition', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()